## Selenium

### Chrome chromedriver
https://chromedriver.storage.googleapis.com/index.html?path=2.28/
Unzip downlaoded chromedriver.zip
sudo mv ~/Downloads/chromedriver /usr/local/bin/

### Firefox geckodriver
https://github.com/mozilla/geckodriver/releases
tar xzvf gekodriver-v0.19.1-linux64.tar.gz
sudo mv ~/Downloads/geckodriver /usr/local/bin/

In [ ]:
!pip install splinter

In [ ]:
#refer to https://python-forum.io/Thread-facebook-friends-crawler
#refer to http://blog.munhou.com/facebook-peng-you-da-tou-tie-pa-chong/
#Have to add time sleep on click and scrolling for human behavior simulation

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
#from selenium.webdriver.support import expected_conditions as EC
#from selenium.webdriver.support.ui import WebDriverWait
import time


your_email = ""
your_password = ""

browser = webdriver.Firefox(executable_path='/usr/local/bin/geckodriver')
#browser = webdriver.Chrome(executable_path='/usr/local/bin/chromedriver')

def facebook():
    CLICK_PAUSE_TIME = 0.5
    # login fb
    browser.get('https://www.facebook.com')
    browser.find_element_by_name('email').send_keys(your_email)
    browser.find_element_by_name('pass').send_keys(your_password)
    elem = browser.find_element_by_id('loginbutton')
    time.sleep(CLICK_PAUSE_TIME)
    elem.click()
    time.sleep(CLICK_PAUSE_TIME)
    get_friend_list()

def get_friend_list():
    CLICK_PAUSE_TIME = 0.5
     # go to profile page
    elem = browser.find_element_by_xpath('//a[@title="個人檔案"]')
    #elem = browser.find_element_by_xpath('//a[@title="Profile"]')
    time.sleep(CLICK_PAUSE_TIME)
    elem.click()
    time.sleep(CLICK_PAUSE_TIME)

    # go to friend page
    elem = browser.find_element_by_xpath('//a[@data-tab-key="friends"]')
    time.sleep(CLICK_PAUSE_TIME)
    elem.click()
    time.sleep(CLICK_PAUSE_TIME)

    # click on all friends tab
    elem = browser.find_element_by_xpath('//a[@name="所有朋友"]')
    #elem = browser.find_element_by_xpath('//a[@name="All friends"]')
    link = elem.get_attribute('href')
    browser.get(link)

    # scroll to bottom
    scroll_to_bottom()

    # get all friend profile page url
    elems = browser.find_elements_by_xpath('//div[@class="uiProfileBlockContent"]/div/div/div/a')
    profile_pages = [elem.get_attribute('href') for elem in elems]
    print(profile_pages)

def scroll_to_bottom():
    SCROLL_PAUSE_TIME = 0.5
    # Get scroll height
    last_height = browser.execute_script("return document.body.scrollHeight")
 
    while True:
        # Scroll down to bottom
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
 
        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)
 
        # Calculate new scroll height and compare with last scroll height
        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
        
facebook()